In [1]:
## \\satpaeb51000155.inedc.corpintra.net\Global_ATS_Data\05.Electic_Vehicle_Data\KFE\02_Members\13_Anil\Axle_Failure_Prediction

# !pip install pandas


In [2]:
import pandas as pd 
import numpy as np
import os 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
import datetime

In [4]:
df = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_NotFailed_Cases\216_Dec23_to_March23_20NotFailed.xlsx")


FileNotFoundError: [Errno 2] No such file or directory: 'W:\\KFE\\02_Members\\08_Ganesh\\Lakshmi-NN\\Current\\Data_NotFailed_Cases\\216_Dec23_to_March23_20NotFailed.xlsx'

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
cols = df.columns
df = df[(df['BM03_IsoRes_Cval']<65535)&(df['EMD1_ACT_MotorTemperature']<150)&(df['Ax2RiFixBearTemp']<110)&(df['EMD3_ACT_MotorTemperature']<110)]
df[cols].max()
df.set_index('t',inplace=True)

In [ ]:
df.drop(columns=['Date-Time','EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)
cols = df.columns

In [ ]:
df['BM03_IsoRes_Cval'].max()

In [ ]:
df2 = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_FailedCases\216_March23_to_June23_20FailedM3.xlsx")

In [ ]:

df2 = df2[(df2['BM03_IsoRes_Cval']<65535)&(df2['EMD1_ACT_MotorTemperature']<150)&(df2['Ax2LeFixBearTemp']<110)&(df2['EMD3_ACT_MotorTemperature']<110)]
df2[cols].max()
df2.set_index('t',inplace=True)

In [ ]:
df3 = df2[df2['Date-Time']<=datetime.datetime.strptime("2020-05-31","%Y-%m-%d")]

In [ ]:
df5 = df2[df2['Date-Time']>datetime.datetime.strptime("2020-05-31","%Y-%m-%d")]

In [ ]:
df6 = df5.copy()

In [ ]:
df5.drop(columns=['Date-Time'],inplace=True)

In [ ]:
len(df5.columns)

In [ ]:
X_train, X_test, = df,df5
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), 
                              columns=X_train.columns, 
                              index=X_train.index)
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)

In [ ]:
X_train.columns 

In [ ]:
# import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras import regularizers

In [ ]:
act_func = 'relu'

# Input layer:
model=Sequential()
# First hidden layer, connected to input vector X. 
model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.0),
                input_shape=(X_train.shape[1],)
               )
         )

model.add(Dense(2,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(X_train.shape[1],
                kernel_initializer='glorot_uniform'))

model.compile(loss='mse',optimizer='adam')

# Train model for 100 epochs, batch size of 10: 
NUM_EPOCHS=100
BATCH_SIZE=10

In [ ]:
history=model.fit(np.array(X_train),np.array(X_train),
                  batch_size=BATCH_SIZE, 
                  epochs=NUM_EPOCHS,
                  validation_split=0.05,
                  verbose = 1)

In [ ]:
plt.plot(history.history['loss'],
         'b',
         label='Training loss')
plt.plot(history.history['val_loss'],
         'r',
         label='Validation loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss, [mse]')
plt.show()

# ZZ0216 Failed in June 23 and Model predicts that in June 1st

In [ ]:
X_pred = model.predict(np.array(X_train))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_train.columns)
X_pred.index = X_train.index

base = pd.DataFrame(index=X_train.index)
base['Loss_mae'] = np.mean(np.abs(X_pred-X_train), axis = 1)
plt.figure()
sns.distplot(base['Loss_mae'],
             bins = 10, 
             kde= True,
            color = 'blue');


In [ ]:
base['Loss_mae'].mean()+1.96*base['Loss_mae'].std()

In [ ]:
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.28
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']

In [ ]:
sns.distplot(scored['Loss_mae'],
             bins = 10, 
             kde= True,
            color = 'blue');

In [ ]:
scored['Loss_mae'].mean()+1.96*scored['Loss_mae'].std()

In [ ]:
scored['Time'] = scored.index
scored.head()

In [ ]:
scored.drop(columns=['Time'],inplace=True)

In [ ]:
X_pred_train = model.predict(np.array(X_train))
X_pred_train = pd.DataFrame(X_pred_train, 
                      columns=X_train.columns)
X_pred_train.index = X_train.index

scored_train = pd.DataFrame(index=X_train.index)
scored_train['Loss_mae'] = np.mean(np.abs(X_pred_train-X_train), axis = 1)
scored_train['Threshold'] = 0.24
scored_train['Anomaly'] = scored_train['Loss_mae'] > scored_train['Threshold']
scored = pd.concat([scored_train, scored])

In [ ]:
#scored.plot(logy=True,  figsize = (10,6), color = ['blue','red'])

In [ ]:
scored['Loss_mae'].max()

In [ ]:
failure_time_start = scored[scored['Anomaly']==True].head().index.tolist()[0]-datetime.timedelta(hours=2)
failure_time_end = scored[scored['Anomaly']==True].head().index.tolist()[0]+datetime.timedelta(hours=2)

In [ ]:
failed_data  = scored[(scored['Time']>=failure_time_start)&(scored['Time']<=failure_time_end)]

In [ ]:
import seaborn as sns


In [ ]:

ax = sns.lineplot(data=failed_data,x='Time',y='Loss_mae')
ax.plot(failed_data['Threshold'],'red')



In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig = make_subplots(rows=1, cols=1,subplot_titles=['e-Axle Failure Prediction'])

fig.add_trace(
    go.Scatter(
            y=failed_data['Loss_mae'],
            x=failed_data['Time'],
            name='Loss_MAE',
            ),row=1,col=1)
fig.add_trace(go.Line( x=failed_data["Time"], y=failed_data["Threshold"],name='Threshold',line={'color': 'red'}))
    
fig.update_layout(autosize=False,width=1892,height=881,title={'text': "ZZ0216_23rd_June",
                                    'y':.95,'x':0.5,'xanchor': 'center','yanchor': 'top'})

fig.write_html(r"C:\Users\PID6420\Downloads\Axle_Failure_Prediction\ZZ0216_23rd_June.html", auto_open=False)


# Motor 3 Failed for ZZ0216 on January 4th Model Predicts that it on 8th Decemeber

In [ ]:

dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_FailedCases\216_Oct4_to_Jan4_21FailedM3.xlsx")

In [ ]:
X_test1 = dfNew[dfNew['Date-Time']>=datetime.datetime.strptime("2020-12-04","%Y-%m-%d")]

In [ ]:
X_test = X_test1.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)

In [ ]:
X_test.head()

In [ ]:
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']

In [ ]:
scored['Time'] = X_test1['Date-Time']

In [ ]:
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

# ZZ0224 Failed on August Model Predicted on 8th July .

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_FailedCases\224_May_to_AugFailedM3.xlsx")

In [ ]:
X_test1 = dfNew[dfNew['Date-Time']>=datetime.datetime.strptime("2020-07-01","%Y-%m-%d")]

In [ ]:
X_test = X_test1.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)

In [ ]:
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.24
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']

In [ ]:
scored['Time'] = X_test1['Date-Time']

In [ ]:
len(scored[scored['Anomaly']==True]['Time'])

In [ ]:
scored['Loss_mae'].max()

#  ZZ0222: Good and Model : Good 

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_NotFailed_Cases\222_Jan13_to_April13_20NotFailed.xlsx")

In [ ]:
X_test = dfNew.copy()
X_test.drop(columns=['Date-Time','t',],inplace=True)

In [ ]:
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)

In [ ]:
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)

In [ ]:
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']

In [ ]:
scored['Time'] = dfNew['Date-Time']

In [ ]:
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

#  ZZ0221: Good and Model : bad 

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_NotFailed_Cases\221_Aug05_to_Nov5_19NotFailed.xlsx")

In [ ]:
X_test = dfNew.copy()
X_test.drop(columns=['Date-Time','t',],inplace=True)

In [ ]:
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)

In [ ]:
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']

In [ ]:
scored['Time'] = dfNew['Date-Time']

In [ ]:
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

In [ ]:
dfNew[cols].max()

# ZZ0221 failed on Feb 5 model predcited on Jan 15th 

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_FailedCases_Recent\221_Nov05_to_Feb5_20FailedM3.xlsx")

In [ ]:
X_test1 = dfNew[dfNew['Date-Time']>=datetime.datetime.strptime("2019-11-01","%Y-%m-%d")]

In [ ]:
X_test = X_test1.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)

In [ ]:
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)

In [ ]:
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.24
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']

In [ ]:
scored['Time'] = X_test1['Date-Time']

In [ ]:
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

# ZZ0214 Failed on June 30 Model  predicted on 18th june

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_FailedCases_Recent\214_March_to_June30_20FailedM3.xlsx")

In [ ]:
X_test1 = dfNew[dfNew['Date-Time']>=datetime.datetime.strptime("2020-06-01","%Y-%m-%d")]
X_test = X_test1.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored['Time'] = X_test1['Date-Time']
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

# ZZ0213 Failed on Feb 20 Model Predcited on Jan21st

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_FailedCases_Recent\213_Nov18_to_Feb18_20FailedM1.xlsx")

In [ ]:
X_test1 = dfNew[dfNew['Date-Time']>=datetime.datetime.strptime("2020-01-18","%Y-%m-%d")]
X_test = X_test1.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored['Time'] = X_test1['Date-Time']
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

# ZZ0211 Failed on June 30 Model predicted on 18th May.

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_FailedCases_Recent\211_March18_to_June18_20FailedM3.xlsx")

In [ ]:
dfNew.head()

In [ ]:
X_test1 = dfNew[dfNew['Date-Time']>=datetime.datetime.strptime("2020-03-18","%Y-%m-%d")]
X_test = X_test1.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored['Time'] = X_test1['Date-Time']
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

# Good case ZZ0211  Model : Good

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_NotFailed_Cases\211_Dec18_to_March18_20NotFailed.xlsx")

In [ ]:

X_test = dfNew.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored['Time'] = dfNew['Date-Time']
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

# Good case ZZ0213  Model : Good 

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_NotFailed_Cases\216_Dec23_to_March23_20NotFailed.xlsx")

In [ ]:
X_test = dfNew.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored['Time'] = dfNew['Date-Time']
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

# Good case ZZ0216  Model : Bad

In [ ]:
dfNew = pd.read_excel(r"W:\KFE\02_Members\08_Ganesh\Lakshmi-NN\Current\Data_NotFailed_Cases\216_July4_to_Oct4_20NotFailed.xlsx")

In [ ]:
dfNew = dfNew[dfNew['BM03_IsoRes_Cval']<65535]
dfNew = dfNew[dfNew['Ax2RiFixBearTemp']<100]                   

In [ ]:
X_test = dfNew.copy()
X_test.drop(columns=['Date-Time','t'],inplace=True)
X_test.drop(columns=['EMD1_ACT_InverterState',
 'EMD2_ACT_InverterState',
 'EMD3_ACT_InverterState',
 'EMD4_ACT_InverterState'],inplace=True)
X_test = pd.DataFrame(scaler.fit_transform(X_test), 
                              columns=X_test.columns, 
                              index=X_test.index)
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.26
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored['Time'] = dfNew['Date-Time']
scored[scored['Anomaly']==True]['Time'].head()

In [ ]:
scored['Loss_mae'].max()

In [ ]:
len(scored[scored['Anomaly']==True]['Time'])

In [ ]:
cols = ['AccelPdlPosn_Cval', 'Ax1LeFixBearTemp', 'Ax1RiFixBearTemp',
       'Ax2LeFixBearTemp', 'Ax2RiFixBearTemp', 'BM03_IsoRes_Cval', 'Date-Time',
       'EMD1_ACT_InverterState', 'EMD1_ACT_MotorSpeed',
       'EMD1_ACT_MotorTemperature', 'EMD1_ACT_Torque',
       'EMD2_ACT_InverterState', 'EMD2_ACT_MotorSpeed',
       'EMD2_ACT_MotorTemperature', 'EMD2_ACT_Torque',
       'EMD3_ACT_InverterState', 'EMD3_ACT_MotorSpeed',
       'EMD3_ACT_MotorTemperature', 'EMD3_ACT_Torque',
       'EMD4_ACT_InverterState', 'EMD4_ACT_MotorSpeed',
       'EMD4_ACT_MotorTemperature', 'EMD4_ACT_Torque', 'StwAngl_Cval_SWS',
       'VehSpd_Cval_CPC']
dfNew[dfNew['Date-Time']==datetime.datetime.strptime("2020-07-22 17:35:04",'%Y-%m-%d %H:%M:%S')][cols].values.tolist()

In [ ]:
dfNew[['AccelPdlPosn_Cval', 'Ax1LeFixBearTemp', 'Ax1RiFixBearTemp',
       'Ax2LeFixBearTemp', 'Ax2RiFixBearTemp', 'BM03_IsoRes_Cval', 'Date-Time',
       'EMD1_ACT_InverterState', 'EMD1_ACT_MotorSpeed',
       'EMD1_ACT_MotorTemperature', 'EMD1_ACT_Torque',
       'EMD2_ACT_InverterState', 'EMD2_ACT_MotorSpeed',
       'EMD2_ACT_MotorTemperature', 'EMD2_ACT_Torque',
       'EMD3_ACT_InverterState', 'EMD3_ACT_MotorSpeed',
       'EMD3_ACT_MotorTemperature', 'EMD3_ACT_Torque',
       'EMD4_ACT_InverterState', 'EMD4_ACT_MotorSpeed',
       'EMD4_ACT_MotorTemperature', 'EMD4_ACT_Torque', 'StwAngl_Cval_SWS',
       'VehSpd_Cval_CPC']].max()